# Assignment Link Generator w/ `git`

## Overview

The following steps take place when the user clicks on a link when generates a launch request:

1. A user session is initiated with the LTI 1.3 login initation flow.
2. The git repository referred to in the link is cloned, fetched, and merged into the user's home directory and, optionally, a target directory within the user's home directory.
3. If the repo is private (set with the `is_private` option), then the service with authenticate with the private repo on the user's behalf using the repo's valid username and token.
3. If a target path (`targetPath`) is specified, the repo is cloned into the specified target path.
4. The `depth` specifies the number of commits in the `git history` to clone.
5. The `branch` specifies the git source branch to fetch from.
5. If a file path is added to the `urlpath` argument, then the user is redirected to the file specified by the path and file name.


> ### NOTE: Canvas LMS: Assignment Links vs Custom Fields

> The Canvas LMS expects the assignment link to include URL encoded parameters since the request is sent to the External Tool as a POST request (in this case IllumiDesk is the External Tool). However, all characters after the domain and `next=` part should be encoded, even those that are usually considered safe, such as `/`, `&`, and `=`.

> The `Custom Fields` text box in the `App -> Settings` section, on the other hand, does not expect all characters to be URL encoded. The `/` characters that are assigned as query parameter values should be encoded, but not the `&` and `=` characters.

## Usage

### Toggles

- `is_assignment_link`: by checking the `is_assignment_link` the link is created with fully encoded values including the `&` and `=` characters. Some LMS's, such as the Canvas LMS, require assignment links encoded this way.
- `is_jupyterlab`: by toggling the `is_jupyterlab` checkbox the user will either open a `Jupyter Lab` workspace or, if unchecked, open the `Jupyter Classic` interface.
- `is_private`: when checked, this allows users to add their username and git token to clone content located in private repos.

### Fields

- **Branch (branch)**: the branch the git/clone/merge steps should reference. Defaults to `master`.
- **Repository URL (repo_url)**: git repo source URL or folder. If you enter a folder name, then this folder should be accesible from the student's environment, such as the `shared` directory.
- **URL Path (url_path)**: The URL path the user should navigate after completing the git fetch/merge task. If a folder and file are specified, then the user will open the `*.ipynb` file itself.

Only applicable if **is_private** is `True` (checked):

- **Git Username (git_username)**: the git username. Applicable only with the repo is private.
- **Git Password (git_password)**: the git password. Applicable only with the repo is private.

In [2]:
import os
from ipywidgets import interact
from urllib.parse import urlunparse, urlparse, urlencode, parse_qs, parse_qsl, quote
from IPython.display import Markdown


@interact
def make_launch_link(is_assignment_link=True, is_jupyterlab=False, is_private=False, git_username='<only applicable if is_private>', git_token='<only applicable if is_private>', branch='master', illumidesk_url='https://flatiron.illumidesk.com', repo_url='https://github.com/foobar/myrepo', urlpath='myrepo/notebook_01.ipynb'):
    """
    Generate a launch request which clones and merges source files from a git-based
    repository.
    """

    # Parse the query to its constituent parts
    domain_scheme, domain_netloc, domain_path, domain_params, domain_query_str, domain_fragment = urlparse(illumidesk_url.strip())
    
    repo_scheme, repo_netloc, repo_path, repo_params, repo_query_str, repo_fragment = urlparse(repo_url.strip())
    folder_from_repo_url_path = os.path.basename(os.path.normpath(repo_path))
    
    # Make sure the path doesn't contain multiple slashes
    if not domain_path.endswith('/'):
        domain_path += '/'
    domain_path += 'user-redirect/git-pull'
    
    # With Canvas using LTI 11 Assignment launch requests all characters after the netloc are considered unsafe.
    # When adding custom parameters within the App Settings -> Custom Fields section, only items after the 
    path_encoded = ''
    if is_assignment_link:
        path_encoded = quote(domain_path, safe='')
    else:
        path_encoded = quote(domain_path)

    path_redirect_url = f'next={path_encoded}'
    assignment_link_path = f'/hub?next={path_encoded}'
    
    # Create a tuple of query params from original domain link
    query_params_from_illumidesk_url = parse_qsl(domain_query_str, keep_blank_values=True)
    
    # Set path based on whether or not the user would like to spawn JupyterLab or Jupyter Classic
    urlpath_workspace = ''
    if is_jupyterlab:
        urlpath_workspace = f'lab/tree/{folder_from_repo_url_path}/{urlpath}?autodecode'
    else:
        urlpath_workspace = f'tree/{folder_from_repo_url_path}/{urlpath}'
    
    # Create a tuple of query params for git functionality. Check whether or not we want to launch with
    # jupyterlab to add additional items to the path.
    query_params_for_git = [('repo', repo_url), ('branch', branch), ('urlpath', urlpath_workspace)]
    if is_private:
        query_params_for_git = [('repo', repo_url), (git_username, ''), (git_token, ''), ('branch', branch), ('urlpath', urlpath_workspace)]
    
    # Merge query params into one list of tuples
    query_params_all = query_params_from_illumidesk_url + query_params_for_git
    
    # First build urlencoded query params where the &, =, and / are considered safe. Then, percent encode
    # all characters.
    encoded_query_params = urlencode(query_params_all)
    encoded_query_params_without_safe_chars = quote(urlencode(query_params_all), safe='')
    
    assignment_link_url = urlunparse((domain_scheme, domain_netloc, assignment_link_path, domain_params, encoded_query_params_without_safe_chars, domain_fragment))
    
    path_url = urlunparse(('', '', path_redirect_url, domain_params, encoded_query_params, domain_fragment))
    if is_assignment_link:
        return assignment_link_url
    return path_url


interactive(children=(Checkbox(value=True, description='is_assignment_link'), Checkbox(value=False, descriptio…